# 10 `for` loops

In this lesson we will learn how to extract geospatial data from a CSV to create a `geopandas.GeoDataFrame` and go over some more customizations for maps and matplotlib figures.

<!--
TODO: learning objectives
-->

## About the data

The U.S. energy landscape relies on a mix of fossil fuels and renewables, each with unique environmental and economic impacts. As the nation works toward sustainability and energy security, understanding this energy mix is essential for informed policy and progress toward cleaner energy.

In this lesson, we will use data from the U.S. [Energy Information Administration (EIA)](https://www.eia.gov) about [operable electric generating plants in the United States by energy source, as of May 2023](https://atlas.eia.gov/datasets/eia::power-plants/about). The dataset includes information on plant types and energy sources, offering insights into the diversity of power sources—from fossil fuels to renewables—that supply electricity nationwide. The dataset's metadata can be [accessed here](https://eia.maps.arcgis.com/sharing/rest/content/items/bf5c5110b1b944d299bb683cdbd02d2a/info/metadata/metadata.xml?format=default&output=html)
<!-- TODO: add citation -->
<!-- TODO: add image -->
The EIA data on electric plants has been downloaded as a CSV and reprojected into the EPSG:4269 CRS for this lesson. It can be accessed [here](https://github.com/carmengg/eds-220-book/blob/main/data/power_plants_epsg4269.csv).

<!-- TODO: column descriptions: lat lon, -->

Additionally, we will use a [TIGER shapefile of the US states from the United States Census Bureau](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2022.html#list-tab-790442341). 
[TIGER](https://en.wikipedia.org/wiki/Topologically_Integrated_Geographic_Encoding_and_Referencing) stands for Topologically Integrated Geographic Encoding and Referencing. This used to be the data format the US Census distributed geospatial data, but since 2008 TIGER files are converted to shapefiles. You can view the metadata for all the TIGER shapefiles [here](https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line.html). 

Follow these steps to download shapefile with the United States' states:

1. At the bottom of the [2022 page](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2022.html#list-tab-790442341), under Download, click on "Web Interface"
2. For year, select 2022, and for layer type select "States (and equivalent)". Click submit. 
3. Click on "Download national file".


The column descriptions for the US states shapefile are:

![Source: TIGER/Line Shapefiles Technical Documentation](/book/images/lesson-11/tiger_shp_columns.png)


## CSV to `geopandas.GeoDataFrame`

Let's start by importing packages and updating viewing options:

In [ ]:
import os

import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import geopandas as gpd
import matplotlib.pyplot as plt


# Display all columns when looking at dataframes
pd.set_option("display.max.columns", None)

Next, we import the power plants dataset.  In this lesson, we have downloaded the data into a `data/` folder in the same level as our notebook.

In [ ]:
# Import power plants data
URL = 'https://raw.githubusercontent.com/carmengg/eds-220-book/refs/heads/main/data/power_plants_epsg4269.csv'
power_plants = pd.read_csv(URL)

# Simpify column names
power_plants.columns = power_plants.columns.str.lower()

# Drop first column
power_plants = power_plants.drop(columns='unnamed: 0')

power_plants.head(3)

<!--TO DO: Need to drop the unnamed column from file--> 

The power plants file is a CSV. Unlike shapefiles or other geospatial file formats, `geopandas` doesn't have a way to extract a geometry column from a CSV file, so we will need to create this geometry manually. 

To do so we will use the longitude and latitude columns in the CSV, these indicate the location of the power plants in the NAD83 CRS (EPSG:4269).
We can use this information to create a new `geopandas.GeoDataFrame` from the `pandas.DataFrame` using the `geopandas` function [`points_from_xy()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html):

In [ ]:
# Create points from latitude and longitude
points = gpd.points_from_xy(power_plants.longitude, 
                            power_plants.latitude)

# Create geodataframe
power_plants = gpd.GeoDataFrame(power_plants,    # Data
                                geometry=points, # Specify geometry column
                                crs='EPSG:4269'  # Specify CRS
                                )

Let's check that we now have a `geometry` column:

In [ ]:
power_plants['geometry']

With the `geometry` column and CRS, we can plot our dataset:

In [ ]:
power_plants.plot()  

## f-strings

So far, we have printed variables using string concatenation inside the `print()` function. 
This means that we write commas between every string and variable we want to print, and then the `print()` function concatenates these into a single string. For example:

In [ ]:
print('CRS: ', power_plants.crs)

Another popular way of mixing strings and variables in print statements is by creating an **f-string** which stands for "formatted string". 
The simplest syntax for an f-string is:
```python
f" some text {replace}"
```
where `replace` can be a variable, an expression, or a function or method call. 
For example:

In [ ]:
# Explore CRS
print(f"ellipsoid: {power_plants.crs.ellipsoid}")
print(f"datum: {power_plants.crs.datum}")

We just created a string replacing the value inside the curly brackets `{}`. 

One of the advantages of using f-strings is that they offer customization for formatting the output:

In [ ]:
# Set the label width to 25 characters, aligning the answers
print(f"{'Is the CRS geographic?:':<25} {power_plants.crs.is_geographic}")
print(f"{'Is the CRS projected?:':<25} {power_plants.crs.is_projected}")

:::{.callout-caution}
## Use f-strings or not?

Whether you use an f-string or simply concatenate strings with variables inside your print statements depends entirely on the application. For quickly checking a variable, a print statement might be enough, while using f-strings can be better to include custom messages during runtime. The best tool can be different depending on the task!

These are some good resources to learn more about f-string formatting:

- [Real Python - Python's F-String for String Interpolation and Formatting](https://realpython.com/python-f-strings/)

- [Python documentation- Format Specification Mini-Language](https://docs.python.org/3/library/string.html#formatspec)
:::

## Import shapefile

Let's import the TIGER shapefile

In [ ]:
# Import states data
fp = os.path.join('data','tl_2022_us_state','tl_2022_us_state.shp')
states = gpd.read_file(fp)

# Simplify column names 
states.columns = states.columns.str.lower()

states.head(3)

and obtain some preliminary geospatial information about the states geodataframe:

In [ ]:
print(states.crs)
states.plot()

## `for` loops

 It can be easier to work with the codes as numbers instead of strings, so let's update the corresponding columns in the states geo-dataframe. We start by checking the data type of the `region`, `division`, and `statefp` columns:

In [ ]:
 code_cols = ['region', 'division', 'statefp']

# Check whether codes columns are strings
 for column in code_cols: 
    print(f"{column} is string dtype? {is_string_dtype(states[column])}")

Remember `for` loops execute a block of code a fixed number of times, iterating over a set of objects. In this case, we iterate over the list of column names `code_cols = ['region', 'division', 'statefp']`.

:::{.callout-caution}
## DRY code
We could have checked whether all the `region`, `division`, and `statefp` columns were of string data type by using the following code:

```python
print(f"region is string dtype? {is_string_dtype(states['region'])}")
print(f"division is string dtype? {is_string_dtype(states['division'])}")
print(f"statefp is string dtype? {is_string_dtype(states['statefp'])}")
```
However, this is inconvenient as it repeats the same pieces of code, only changing the column name. Instead, using the `for` loop allows us to succintly print the same information:
```python
code_cols = ['region', 'division', 'statefp']

for column in code_cols: 
    print(f"{column} is string dtype? {is_string_dtype(states[column])}")
```
**Don't Repeat Yourself (DRY)** is a core programming principle that encourages  reducing redundancy and consolidating repeated logic. Try implementing it as much as possible! If you need to repeat the "same" code more than twice, you likely need a `for` loop.
:::

Next, we update the data type of the code columns to be integers. This time, we check the data type of the column using the `is_numeric_dtype()` function inside an **`assert`** statement:

In [ ]:
# Update code columns into integers
for column in code_cols:
    states[column] = states[column].astype('int')
    assert is_numeric_dtype(states[column])  # Check conversion

The `assert` keyword does nothing if the expression next to it evaluates to `True` and raises an `AssertionError` exception and stops your code form running any further. For example, 

In [ ]:
#| error: true
# Does nothing if statement is True
assert 2+2 == 4

# Raises an error if statement is False
assert 2+2 == 3

In our data type conversion code, since no `AssertionError` was raised, we can be confident that the data type was updated. 

## Data selection

For this lesson, we want to use only the contiguous states. As seen in the plot, the data covers a bigger extension. 

:::{.callout-tip}
## Check-in
From the TIGER shapefiles metadata we know that:
 
> In addition to the fifty states, the Census Bureau treats the District of Columbia, Puerto Rico, and the Island areas (American Samoa, the Commonwealth of the Northern Mariana Islands, Guam, and the U.S. Virgin Islands) as statistical equivalents of states for the purpose of data presentation. 

In [this US Census Bureau file](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf) we can see what each code for the region, division, and state corresponds to. 

- What are the unique values for region, division, or state codes in the data?
- Which codes should should we select to keep only states in the contiguous US?

<!--
 States correspond to regions 1 through 4. 
 However, there's also a region code 9.
 Hawaii = state code 15
 Alaska = satate code 02
 Need to exclude: region code 9 ans state codes 15 and 02
 -->
:::

Let's go ahead and select the data:

In [ ]:
# Select contiguous US states
contiguous = states[(states.region!=9) & (~states.statefp.isin([2,15]))]

In this code we used the syntax 
```python
~df.column.isin([val1, val2, val3])
```
The `~` tilde symbol is used in Python to negate a statement. 
So the previous line could be read as "the values in `df`'s column which are *not* in the list `[val1, val2, val3]`."

::: {.callout-tip}
## Check-in
Select the data in the `power_plants` data frame for the contiguous US states. 
<!--

In [ ]:
power_plants = power_plants[~power_plants.state.isin(['Puerto Rico','Hawaii','Alaska'])]

-->
:::

## Plotting

Before we plot our data, let's make sure they are in the same CRS:

In [ ]:
contiguous.crs == power_plants.crs

In [ ]:
#| code-fold: true

fig, ax = plt.subplots(figsize=(9, 5)) # Update figure size

# Remove the axis for a cleaner map
ax.axis('off')

# Title for the plot
ax.set_title('Operable electric generating plants in the contiguous United States', 
              fontsize=15)

# Add states
contiguous.plot(ax=ax,
               color='none',
               edgecolor='#362312')

# Add electric power plants colored by energy source
power_plants.plot(ax=ax, 
                  column='primsource',
                  legend=True,
                  markersize=4,
                  cmap='tab20',
                  alpha=0.5,
                  legend_kwds={
                      'title': 'Primary energy source',
                      'title_fontsize': 'small',
                      'fontsize': 'small',
                      'loc': 'upper left',
                      'bbox_to_anchor': (0, 0),
                      'ncol': 6  
                  })
                  
plt.show()

In the map above we specified the figure size when creating the plot. This size is given in inches, but [can be updated to other units](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/figure_size_units.html) (pixels, cm, etc). 

We also controlled the legend location using `loc` and `bbox_to_anchor` in the `legend_kwds`:

- `loc` indicates the corner of the legend we want to use for placement, and
- `bbox_to_anchor` is a tuple with coordinates indicating where to place the corner specified in `loc` relative to the axes. Values between 0 and 1 are within the axes.

<!--TODO: add diagram of bbox-->

`matplotlib` uses a variety of ways to locate elements within the graph and it is best to check the documentation to not spend too much time fidling with locations.

<!--

https://www.flexprojector.com

https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-vector-plots/python-customize-map-legends-geopandas/

https://stackoverflow.com/questions/74143732/customize-legend-labels-in-geopandas
-->

## `for` with `zip`

Often, we need to iterate simultaneously over two iterables. 
The `zip()` function in Python allows you to combine two or more lists (or other iterables) so that you can iterate over their elements in pairs. When used with a `for` loop, it lets you process elements from each list together, like this example:


In [ ]:
# Iterate over a single list
numbers = [1, 2, 3]
for num in numbers:
    print(num)

print('\n')  # Blank line

# Iterate over two lists in pairs using zip()
letters = ['a', 'b', 'c']
for num, letter in zip(numbers, letters):
    print(num, letter)

A common situation when code gets repeated is when creating subplots. For example:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(7, 3))

axes[0].set_title('This is axis 0')
axes[1].set_title('This is axis 1')
axes[2].set_title('This is axis 2')

plt.show()

In this example, notice that the `axes` variable returned by the `plt.subplots()` function is actually a list of axes we can iterate over. 

:::{.callout-tip}
## Check-in
Use `for` and `zip()` to create the same subplots and avoid redundancy.
<!--
```python
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(7, 3))
for ax, i in zip(axes, [0,1,2]):
    ax.set_title(f'This is axis {i}')
plt.show()
```
-->
:::

Remember that the figure and the axes are separete elements in a `matplotlib` plot.

![Image source: [Getting Started with Matplotlib](https://www.skytowner.com/explore/getting_started_with_matplotlib)](/book/images/lesson-10/matplotlib_figure_axes_axis.png)


:::{.callout-tip}
## Exercise
1. Select the power plants in California in a variable named `ca_power_plants`.

2. Create a _list_ named `top_sources` with California's top 3 electric primary sources.

3. Isolate the California state boundary in a variable named `ca_boundary`.

4. Recreate the following plot:

In [ ]:
#| echo: false
# Create a list with california top 3 electric primary sources
ca_power_plants = power_plants[power_plants.state=='California']
top_sources = (ca_power_plants['primsource']
                            .value_counts()
                            .index[:3]
                            .tolist()
                            )

# Isolate the CA boundary
ca_boundary = states[states.name=='California']

# Create plot
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6, 3))

for ax, source in zip(axes, top_sources):
    ca_boundary.plot(ax=ax,                
                    color='none',
                    edgecolor='#362312')
    subset = ca_power_plants[ca_power_plants['primsource'] == source]
    subset.plot(ax=ax, markersize=5, alpha=0.5)
    ax.set_title(source)
    ax.axis('off')  # Remove axes for a cleaner look

plt.suptitle("Top 3 energy sources for electric power plants in CA")
plt.tight_layout()
plt.show()

:::

<!--TODO: add functions here-->